# House Price Prediction

```Data/house_price.csv```には、住宅の価格(```SalePrice```)と住宅の関連情報が格納されています。

- このデータを用いて、住宅の価格(```SalePrice```)を予測するモデルを構築、評価しなさい。
- データ中身を確認しながら、適切な特徴量エンジニアリングやモデル選定ための考えもまとめてください。

In [1]:
import pandas as pd
import numpy as np

In [2]:
#データ読み込み
%cd C:\Users\tatum\test\chss_2023_assignment\最終課題\House_Price\Data
df=pd.read_csv("house_price.csv")
display(df)

C:\Users\tatum\test\chss_2023_assignment\最終課題\House_Price\Data


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [5]:
#文字列を数値に変換その
df_nominal=df.select_dtypes(include="O")#pythonオブジェクト型の列を抽出
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()
oe.fit(df_nominal)
df_nominal_transformed=pd.DataFrame(oe.transform(df_nominal))#数値に変換
df_nominal_transformed=df_nominal_transformed.set_axis(oe.get_feature_names_out(),1)#ラベルを文字列に再変換
df_numerical=df.drop(df_nominal,axis=1)
df_numerical=pd.concat([df_numerical,df_nominal_transformed],axis=1)#文字列データを数値変換したデータと入れ替え
display(df_numerical)

C:\Users\tatum\AppData\Local\Temp\ipykernel_24688\419438771.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.set_axis except for the argument 'labels' will be keyword-only.
  df_nominal_transformed=df_nominal_transformed.set_axis(oe.get_feature_names_out(),1)#ラベルを文字列に再変換


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,1.0,1.0,4.0,4.0,2.0,NaN,NaN,NaN,8.0,4.0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,1.0,1.0,4.0,4.0,2.0,NaN,NaN,NaN,8.0,4.0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,1.0,1.0,4.0,4.0,2.0,NaN,NaN,NaN,8.0,4.0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,5.0,2.0,4.0,4.0,2.0,NaN,NaN,NaN,8.0,0.0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,1.0,1.0,4.0,4.0,2.0,NaN,NaN,NaN,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,1.0,1.0,4.0,4.0,2.0,NaN,NaN,NaN,8.0,4.0
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,1.0,2.0,4.0,4.0,2.0,NaN,2.0,NaN,8.0,4.0
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,1.0,1.0,4.0,4.0,2.0,NaN,0.0,2.0,8.0,4.0
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,1.0,2.0,4.0,4.0,2.0,NaN,NaN,NaN,8.0,4.0


In [6]:
#欠損値の補完
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy="median")
imputer.fit(df_numerical)
nullX=imputer.transform(df_numerical)
df_numerical_tr=pd.DataFrame(nullX,columns=df_numerical.columns)
df_numerical_tr

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,1.0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,...,1.0,1.0,4.0,4.0,2.0,1.0,2.0,2.0,8.0,4.0
1,2.0,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,...,1.0,1.0,4.0,4.0,2.0,1.0,2.0,2.0,8.0,4.0
2,3.0,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,...,1.0,1.0,4.0,4.0,2.0,1.0,2.0,2.0,8.0,4.0
3,4.0,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,...,5.0,2.0,4.0,4.0,2.0,1.0,2.0,2.0,8.0,0.0
4,5.0,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,...,1.0,1.0,4.0,4.0,2.0,1.0,2.0,2.0,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,...,1.0,1.0,4.0,4.0,2.0,1.0,2.0,2.0,8.0,4.0
1456,1457.0,20.0,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,...,1.0,2.0,4.0,4.0,2.0,1.0,2.0,2.0,8.0,4.0
1457,1458.0,70.0,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,275.0,...,1.0,1.0,4.0,4.0,2.0,1.0,0.0,2.0,8.0,4.0
1458,1459.0,20.0,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,...,1.0,2.0,4.0,4.0,2.0,1.0,2.0,2.0,8.0,4.0


In [7]:
#正規化
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
scaled_features=scaler.fit_transform(df_numerical_tr)
df_scaled=pd.DataFrame(scaled_features,columns=df_numerical_tr.columns).drop("Id",axis=1)
display(df_scaled)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.122500,0.125089,0.000000,...,0.2,0.5,1.0,1.0,1.0,0.5,0.666667,0.666667,1.0,0.8
1,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.000000,0.173281,0.000000,...,0.2,0.5,1.0,1.0,1.0,0.5,0.666667,0.666667,1.0,0.8
2,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.101250,0.086109,0.000000,...,0.2,0.5,1.0,1.0,1.0,0.5,0.666667,0.666667,1.0,0.8
3,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.000000,0.038271,0.000000,...,1.0,1.0,1.0,1.0,1.0,0.5,0.666667,0.666667,1.0,0.0
4,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.218750,0.116052,0.000000,...,0.2,0.5,1.0,1.0,1.0,0.5,0.666667,0.666667,1.0,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.235294,0.140411,0.030929,0.555556,0.500,0.920290,0.833333,0.000000,0.000000,0.000000,...,0.2,0.5,1.0,1.0,1.0,0.5,0.666667,0.666667,1.0,0.8
1456,0.000000,0.219178,0.055505,0.555556,0.625,0.768116,0.633333,0.074375,0.139972,0.110583,...,0.2,1.0,1.0,1.0,1.0,0.5,0.666667,0.666667,1.0,0.8
1457,0.294118,0.154110,0.036187,0.666667,1.000,0.500000,0.933333,0.000000,0.048724,0.000000,...,0.2,0.5,1.0,1.0,1.0,0.5,0.000000,0.666667,1.0,0.8
1458,0.000000,0.160959,0.039342,0.444444,0.625,0.565217,0.766667,0.000000,0.008682,0.698100,...,0.2,1.0,1.0,1.0,1.0,0.5,0.666667,0.666667,1.0,0.8


In [8]:
#特徴量選択
corr=pd.DataFrame(df_scaled.corr()["SalePrice"]).drop("SalePrice",axis=0)
highcorr=corr[abs(corr["SalePrice"])>=0.5]#SalePriceとの相関係数が0.5以上または-0.5以下の特徴量を抽出
highcorr

,SalePrice
OverallQual,0.790982
YearBuilt,0.522897
YearRemodAdd,0.507101
TotalBsmtSF,0.613581
1stFlrSF,0.605852
GrLivArea,0.708624
FullBath,0.560664
TotRmsAbvGrd,0.533723
GarageCars,0.640409
GarageArea,0.623431


In [9]:
#抽出した特徴量で線形回帰モデルに当てはめ
from sklearn.linear_model import LinearRegression
model=LinearRegression()
from sklearn.model_selection import train_test_split
X=df_scaled.loc[:,highcorr.index]
y=df_numerical_tr["SalePrice"]
assert len(X)==len(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)#分割
model.fit(X_train, y_train)#あてはめ

LinearRegression()

In [10]:
#予測とその評価
from sklearn.metrics import r2_score
y_pred=model.predict(X_test)
r2_score(y_test,y_pred)#R^2の算出

0.8411128435361604